In [ ]:
import spacy
import json
import random
import re
import pandas as pd
import numpy as np
from copy import deepcopy
from sklearn import model_selection
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.symbols import ORTH
from spacy.symbols import NORM
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import tree
import matplotlib.pyplot as plt
import random
import plotly.express as px
import sys 
import fasttext
sys.path.append('./luima_sbd')
import luima_sbd.sbd_utils as luima
from spacy.language import Language
random.seed(42)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          title=None,
                          cmap=plt.cm.Blues):
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(8, 8))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
def top_tfidf_features(row, features, top_n=15):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df


def top_features_in_doc(Xtr, features, row_id, top_n=15):
    ''' Top tfidf features in specific document (matrix row) '''
    xtr_row = Xtr[row_id]
    if type(xtr_row) is not np.ndarray:
        xtr_row = xtr_row.toarray()
    row = np.squeeze(xtr_row)
    return top_tfidf_features(row, features, top_n)


def top_mean_features(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids]
    else:
        D = Xtr
    if type(D) is not np.ndarray:
        D = D.toarray()
    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_features(tfidf_means, features, top_n)


def top_features_by_class(Xtr, y, features, min_tfidf=0.1, top_n=25):
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = {}
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_features(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs[label] = feats_df
    return dfs


def span_top_tfidf(spans_txt, spans_tfidf, features, index):
    print('span text:\n'+spans_txt[index]+'\n')
    print(top_features_in_doc(spans_tfidf, features, index))

In [ ]:
corpus_fpath = './ldsi_s2021/ldsi_bva_sentence_corpus_v1.json'
data = json.load(open(corpus_fpath))
affirmed = open('./ldsi_s2021/affirmed_ids.txt', 'r').read().split("\n")
denied= open('./ldsi_s2021/denied_ids.txt', 'r').read().split("\n")
remanded = open('./ldsi_s2021/remanded_ids.txt', 'r').read().split("\n")
# print(len(affirmed), len(denied), len(remanded))
annotations = data['annotations']
documents_by_id = {d['_id']: d for d in data['documents']}
types_by_id = {t['_id']: t for t in data['types']}
type_ids_by_name = {t['name']: t['_id'] for t in data['types']}
type_names_by_id = {t['_id']: t['name'] for t in data['types']}
doc_id_by_name = {d['name']: d['_id'] for d in data['documents']}
doc_name_by_id = {d['_id']: d['name'] for d in data['documents']}

def make_span_data(documents_by_id, types_by_id, annotations):
    span_data = []
    for a in annotations:
        start = a['start']
        end = a['end']
        document_txt = documents_by_id[a['document']]['plainText']
        atype = a['type']
        document_name=documents_by_id[a['document']]['name']
        if document_name in affirmed:
            decision='affirmed'
        elif document_name in denied:
            decision='denied'
        elif document_name in remanded:
            decision='remanded'
        sd = {'txt': document_txt[start:end],
              'document': a['document'],
              'type': types_by_id[atype]['name'],
              'start': a['start'],
              'start_normalized': a['start'] / len(document_txt),
              'end': a['end'],
              'name': document_name,
              'decisions': decision}
        span_data.append(sd)
    return span_data

spans = make_span_data(documents_by_id, types_by_id, annotations)
span_labels = [s['type'] for s in spans]
span_decisions = [s['decisions'] for s in spans]

In [ ]:
random.seed(42)
aff=random.sample(affirmed, 6)
den=random.sample(denied, 6)
rem=random.sample(remanded, 6)
test_affirm, dev_affirm = aff[0:3], aff[3:6] 
test_denied, dev_denied = den[0:3], den[3:6] 
test_remanded, dev_remanded = rem[0:3], rem[3:6] 

test_ids = test_affirm+test_denied+test_remanded
dev_ids = dev_affirm+dev_denied+dev_remanded

test_spans=[]
dev_spans=[]
train_spans=[]
for s in spans:
    if s['name'] in test_ids:
        test_spans.append(s)
    elif s['name'] in dev_ids:
        dev_spans.append(s)
    else:
        train_spans.append(s)
        
unique_files=pd.DataFrame(train_spans).name.unique()

In [ ]:
token_df = pd.read_pickle("./token_df.pkl")

In [ ]:
%time model = fasttext.train_unsupervised('./fastTextFinalShuffled.txt', epoch=10, minCount=20)

In [ ]:
model.save_model("result/wordEmbeddingsModel.bin")

In [ ]:
model = fasttext.load_model("result/wordEmbeddingsModel.bin")

In [ ]:
len(model.get_words(on_unicode_error='ignore'))

In [ ]:
test_words=["veteran","vet","service","cause","caused","remanded","vietnam","see","board","notice","claim", "judge","records","letter"]
test_word_track=[]
for word in test_words:
    nearest_neigh=model.get_nearest_neighbors(word)
    app_dict={
        "test_word": word,
        "nearest_neighbours": nearest_neigh
    }
    test_word_track.append(app_dict)

In [ ]:
df = pd.DataFrame(test_word_track)
df.to_pickle("./word_emd_NN.pkl")

In [ ]:
test_word_track